We retrieve the results obtained in `graph_fact.py`

In [1]:
import time
import numpy as np
import pickle
with open(r"c:\Users\cacereje\OneDrive\Vanderbilt\Postdoc 2025 Fall\Code\Graph_Fact\result.pkl", "rb") as f:
     result = pickle.load(f)

To determine the factored graph $G$, we just multiply all the matrices in any list of `results`.

In [2]:
import sympy as sp

G = sp.Matrix(np.linalg.multi_dot(result[0]))
G

Matrix([
[1, 0, 1, 0, 0, 0, 0, 0],
[0, 1, 0, 1, 0, 0, 0, 0],
[0, 0, 1, 1, 1, 1, 0, 0],
[0, 0, 0, 0, 1, 0, 1, 0],
[0, 0, 0, 0, 0, 1, 0, 1]])

Using our building block decomposition we form all pairs $(G_{0,n},G_{1,n})$, eliminating isomorphic pairs at every step.

In [ ]:
# no longer needed
# Gf = np.linalg.multi_dot(result[0])
# af_0 = np.ones((5,1))
# eigvals, eigvects = np.linalg.eig(Gf@np.transpose(Gf))
# sf_0_raw = -eigvects[:,0]
# sf_0 = sf_0_raw/(sf_0_raw@af_0)

# def af(n):
#     return (np.linalg.matrix_power(Gf@np.transpose(Gf),n))@af_0

# def sf(n):
#     return sf_0/(sf_0@af(n))

# def norm_lim(A,n):
#     s = sf(n)
#     return (((A@np.transpose(A))@s)@s)/(np.linalg.norm(s)**2)

In [ ]:
# no longer needed
# candidates = []
# tol = 0.5

# for list in result:
#     r = len(list)
#     for i in range(2,r-1):
#         G1f = np.linalg.multi_dot(list[:i])
#         G2f = np.linalg.multi_dot(list[i:])
#         if threshold-tol<norm_lim(G1f,0)<threshold:
#             candidates.append([G1f,G2f])


In [4]:
from graph_fact_tools import *

candidates_abs = []
for list in result:
    r = len(list)
    G1f = list[:1][0]
    G2f = np.linalg.multi_dot(list[1:])
    candidates_abs.append([G1f,G2f])
    G1f = np.linalg.multi_dot(list[:10])
    G2f = list[10:][0]
    candidates_abs.append([G1f,G2f])
    for i in range(2,r-1):
        G1f = np.linalg.multi_dot(list[:i])
        G2f = np.linalg.multi_dot(list[i:])
        candidates_abs.append([G1f,G2f])
    candidates_abs = dedupe_chains(candidates_abs)
print(len(candidates_abs))

35


We obtained precisely $35$ pairs. Now, transform our candidate list to a list of symbolic matrices and compute their norms symbolically.

In [80]:
def pf_norms(pair):
    G1 = sp.Matrix(pair[0])
    G2 = sp.Matrix(pair[1])
    eigs1 = list((G1*(G1.T)).eigenvals().keys())
    eigs2 = list((G2*(G2.T)).eigenvals().keys())
    norm1 = max([ev for ev in eigs1 if ev.is_real])
    norm2 = max([ev for ev in eigs2 if ev.is_real])
    return norm1, norm2

In [170]:
from sympy import latex
from IPython.display import Math

target_norms = (sp.Integer(2),(sp.sqrt(5)+3)/2)
target_norms2 = ((sp.sqrt(5)+3)/2,sp.Integer(2))
pf = [pf_norms(pair) for pair in candidates_abs]
ind = [i for i,x in enumerate(pf) if (x == target_norms or x == target_norms2)]
for i in ind:
    G1 = sp.Matrix(candidates_abs[24][0])
    G2 = sp.Matrix(candidates_abs[24][1])
    print(f"Pair {i+1} has the right norms.\n")
    strings = [latex(vec) for vec in (G1*(G1.T)).eigenvects()[-1][-1]]
    display(Math(r"\text{Basis of eigenspace associated to }\|G_{1}\|^2:"+", ".join(strings)))
    


Pair 17 has the right norms.



<IPython.core.display.Math object>

Pair 25 has the right norms.



<IPython.core.display.Math object>

Neither of these admit a positive eigenvector.

In [ ]:
# from collections import Counter
# from sympy import latex, simplify
# from IPython.display import Math

# # convert to numeric (higher precision) then round for grouping
# pairs_num = []
# rep_sym = {}
# for a_sym, b_sym in norm_list:
#     af = float(sp.N(a_sym, 24))
#     bf = float(sp.N(b_sym, 24))
#     key = (round(af, 8), round(bf, 8))   # adjust precision as needed
#     pairs_num.append(key)
#     rep_sym.setdefault(key, (a_sym, b_sym))

# cnt = Counter(pairs_num)

# rows = []
# for (n1, n2), c in sorted(cnt.items()):
#     a_sym, b_sym = rep_sym[(n1, n2)]
#     prod_sym = latex(sp.together(simplify(a_sym*b_sym)))
#     a_tex = latex(sp.together(simplify(a_sym)))
#     b_tex = latex(sp.together(simplify(b_sym)))
#     rep_tex = f"{a_tex} & {b_tex}"          # math-mode representative
#     rows.append(f"{n1:.8f} & {n2:.8f} & {c:d} & {rep_tex} & {prod_sym}\\\\")

# # header + join
# header = r"\text{Norm 1} & \text{Norm 2} & \text{count} & \|G_{0,n}\|^2 & \|G_{1,n}^t\|^2 & \text{Product}\\ \hline"
# body = "\n".join(rows)
# array_tex = r"\begin{array}{r r r c c c}" + "\n" + header + "\n" + body + "\n" + r"\end{array}"

# display(Math(array_tex))

<IPython.core.display.Math object>